In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50

In [3]:
accepted = pd.read_csv('accepted_2007_to_2018Q4.csv')

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
accepted.drop(index= accepted.loc[accepted['id'].apply(lambda s: len(str(s))) > 10, :].index, inplace = True)

In [5]:
accepted['issue_d'] = accepted['issue_d'].apply(lambda s: pd.to_datetime(s, format='%b-%Y'))

In [17]:
accepted2 = accepted[accepted['issue_d'] < '2015-01-01']

In [19]:
# Fill NA's
accepted2.loc[:, 'pub_rec_bankruptcies'] = accepted2['pub_rec_bankruptcies'].fillna(value = 0)

In [20]:
from dateutil.relativedelta import relativedelta
from datetime import timedelta

def increment_months(s, length):
    while length >0:
        if s.month == 12:
            s = s + timedelta(days = 31)
        else:
            s = s + relativedelta(months = 1)
        length -= 1
    return s

accepted2.loc[~accepted2['last_pymnt_d'].isnull(), 'last_pymnt_d'] = \
    accepted2.loc[~accepted2['last_pymnt_d'].isnull(), 'last_pymnt_d'].apply(lambda s: pd.to_datetime(s, format='%b-%Y'))

accepted2['last_pymnt_d'] = accepted2['last_pymnt_d'].astype(np.datetime64)

a = accepted2.loc[accepted2['last_pymnt_d'].isnull(), ['issue_d', 'total_pymnt', 'installment']].apply(lambda s: increment_months(s[0],np.floor(s[1]/s[2])), axis = 1)

accepted2.loc[accepted2['last_pymnt_d'].isnull(), 'last_pymnt_d'] = a

accepted2['last_pymnt_d'] = accepted2['last_pymnt_d'].astype(np.datetime64)

a = accepted2['last_credit_pull_d'].apply(lambda s: pd.to_datetime(s, format = '%b-%Y'))

accepted2.loc[:, 'last_credit_pull_d'] = a

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
accepted2.iloc[0,:]

id                                                                                     36805548
member_id                                                                                   NaN
loan_amnt                                                                                 10400
funded_amnt                                                                               10400
funded_amnt_inv                                                                           10400
term                                                                                  36 months
int_rate                                                                                   6.99
installment                                                                              321.08
grade                                                                                         A
sub_grade                                                                                    A3
emp_title                               

In [23]:
accepted2.isnull().sum()[(accepted2.isnull().sum() >0) &\
                              (accepted2.isnull().sum() < 10000)]

Series([], dtype: int64)

In [22]:
temp = accepted2.isnull().sum()[(accepted2.isnull().sum() >0) &\
                              (accepted2.isnull().sum() < 10000)].index

accepted2.drop(index = accepted2.loc[accepted2[temp].isnull().any(axis = 1), :].index, inplace = True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
a = accepted2.groupby(['grade','sub_grade'])['mths_since_last_delinq'].agg('median')

accepted2.loc[accepted2['mths_since_last_delinq'].isnull(),'mths_since_last_delinq'] = \
accepted2.loc[accepted2['mths_since_last_delinq'].isnull(), ['grade','sub_grade', 'mths_since_last_delinq']].apply(lambda s: a[s[0], s[1]], axis = 1)

In [26]:
#drop columns that have more than 70% NA's
mask = accepted2.isnull().sum()[(accepted2.isnull().sum() >0)]/accepted2.shape[0]*100 > 70
drop_columns = accepted2.isnull().sum()[(accepted2.isnull().sum() >0)][mask].index

accepted2.drop(columns = drop_columns, inplace = True)

In [53]:
accepted2.isnull().sum().sort_values(ascending = False)

debt_settlement_flag          0
initial_list_status           0
fico_range_low                0
fico_range_high               0
inq_last_6mths                0
mths_since_last_delinq        0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
out_prncp                     0
delinq_2yrs                   0
out_prncp_inv                 0
total_pymnt                   0
total_pymnt_inv               0
total_rec_prncp               0
total_rec_int                 0
total_rec_late_fee            0
recoveries                    0
collection_recovery_fee       0
last_pymnt_d                  0
earliest_cr_line              0
dti                           0
disbursement_method           0
emp_length                    0
loan_amnt                     0
funded_amnt                   0
funded_amnt_inv               0
term                          0
int_rate                      0
installm

In [28]:
accepted2.loc[:, 'emp_length'].fillna(value = '2 years', inplace = True)
accepted2.loc[:, 'emp_title'].fillna(value = 'not_provided', inplace = True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [49]:
for col in accepted2.isnull().sum()[accepted2.isnull().sum() > 0].index:
    a = accepted2.groupby(['grade','sub_grade'])[col].agg('median')
    accepted2.loc[accepted2[col].isnull(), col] = \
    accepted2.loc[accepted2[col].isnull(), ['grade','sub_grade', col]].apply(lambda s: a[s[0], s[1]], axis = 1)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
# Installment is calculated by the following formula
def installment(loan_amnt, int_rate, term):
    r = int_rate/1200
    return loan_amnt*r/(1-(1+r)**(-term))

$$installment = \frac{r PV}{1 - (1+r)^{-term}}$$

In [51]:
accepted2.to_csv('accepted_before_2015.csv', index = False)